## Installations

In [1]:
"""
conda create -n CUDA_ENV python=3.10
conda activate CUDA_ENV
conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
python -m pip install "tensorflow==2.10"
conda install "numpy<2"
"""

'\nconda create -n CUDA_ENV python=3.10\nconda activate CUDA_ENV\nconda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0\npython -m pip install "tensorflow==2.10"\nconda install "numpy<2"\n'

## Imports

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
import numpy as np
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Enabling GPU

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])

    print(tf.config.experimental.get_device_details(gpus[0])['device_name'])

NVIDIA GeForce RTX 3060 Ti


## Getting database

In [4]:
model_path = "All-Animals.keras"
model_exists = os.path.exists(model_path)

retrain = False
dataset = "All-Animals/"
train = f"{dataset}Train"
valid = f"{dataset}Valid"
test = f"{dataset}Test"

## Recreating Dataset

In [5]:
augmented_root = f"{dataset}Augmented"
recreate = not os.path.isdir(augmented_root)

if recreate:
    for path in [train, valid, test]:
        for root, dirs, files in os.walk(path):
            for index, file in enumerate(files):
                if file.lower().endswith(('.jpg', '.jpeg')):
                    old_path = os.path.join(root, file)
                    with Image.open(old_path) as img:
                        img = img.convert("RGB")
                        new_filename = f"{os.path.basename(root)}_{index + 1}.jpeg"
                        new_path = os.path.join(root, new_filename)
                        img.save(new_path, 'JPEG')
                    os.remove(old_path)

## Augmenting pictures

In [6]:
if recreate:
    for root, dirs, files in os.walk(train):
        parent_folder = os.path.basename(root)
        augmented_folder = os.path.join(augmented_root, parent_folder)
        os.makedirs(augmented_folder, exist_ok=True)

        for file in files:
            path = os.path.join(root, file.lower())
            if '_aug_' not in path:
                img = np.array(Image.open(path))
                datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.3, horizontal_flip=True, brightness_range=[0.8, 1.2])
                augmented_images = [next(datagen.flow(np.expand_dims(img, 0), batch_size=1))[0] for _ in range(100)]
                to = 4 if path.endswith(".jpg") else 5 if path.endswith(".jpeg") else 1
                for i, aug_img in enumerate(augmented_images):
                    aug_img_path = os.path.join(augmented_folder, f"{file[:-to]}_aug_{i+1}.jpeg")
                    Image.fromarray(aug_img.astype("uint8")).save(aug_img_path)

train = augmented_root


KeyboardInterrupt



## Dataset Loading

In [49]:
if recreate:
    img_size = (256, 256)
    batch_size = 32

    train_ds = image_dataset_from_directory(train, image_size=img_size, batch_size=batch_size)
    val_ds = image_dataset_from_directory(valid, image_size=img_size, batch_size=batch_size)
    test_ds = image_dataset_from_directory(test, image_size=img_size, batch_size=batch_size)

    class_names = train_ds.class_names
    print(class_names)

NotFoundError: Could not find directory Sea-Animal/Augmented

## Model Definition

In [35]:
with tf.device('/GPU:0'):
    if not retrain:
        model = keras.Sequential([
            layers.Rescaling(1./255, input_shape=(256, 256, 3)),
            layers.Conv2D(32, (3, 3), activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D(),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(len(class_names), activation='softmax')
        ])
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    epochs = 20
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
    model.save(model_path)

Epoch 1/20
13397/13397 [==============================] - 748s 53ms/step - loss: 1.1874 - accuracy: 0.6127 - val_loss: 2.2715 - val_accuracy: 0.5390
Epoch 2/20
13397/13397 [==============================] - 711s 53ms/step - loss: 0.5772 - accuracy: 0.8093 - val_loss: 2.1253 - val_accuracy: 0.6667
Epoch 3/20
13397/13397 [==============================] - 710s 53ms/step - loss: 0.3338 - accuracy: 0.8891 - val_loss: 3.0515 - val_accuracy: 0.6596
Epoch 4/20
13397/13397 [==============================] - 709s 53ms/step - loss: 0.2335 - accuracy: 0.9240 - val_loss: 3.7774 - val_accuracy: 0.6383
Epoch 5/20
13397/13397 [==============================] - 709s 53ms/step - loss: 0.1854 - accuracy: 0.9405 - val_loss: 3.9495 - val_accuracy: 0.6667
Epoch 6/20
13397/13397 [==============================] - 708s 53ms/step - loss: 0.1651 - accuracy: 0.9486 - val_loss: 4.1759 - val_accuracy: 0.6596
Epoch 7/20
13397/13397 [==============================] - 708s 53ms/step - loss: 0.1477 - accuracy: 0.9557

## Test

In [42]:
loss, accuracy = model.evaluate(val_ds)
print(f'Test Accuracy: {accuracy:.4f}')

predictions = model.predict(test_ds)
pred_labels = np.argmax(predictions, axis=1)

true_labels = np.concatenate([y.numpy() for _, y in test_ds], axis=0)

misclassified_indices = np.where(pred_labels != true_labels)[0]

misclassified_data = [(true_labels[idx], pred_labels[idx]) for idx in misclassified_indices[:10]]

print(misclassified_data)


5/5 [==============================] - 0s 17ms/step - loss: 10.1478 - accuracy: 0.6312
Test Accuracy: 0.6312
7/7 [==============================] - 0s 14ms/step
[(6, 4), (4, 12), (2, 12), (10, 12), (6, 3), (10, 8), (6, 12), (8, 10), (6, 10), (6, 9)]
